
<a id='timeseries'></a>
{{ header }}

# Time series / date functionality

pandas contains extensive capabilities and features for working with time series data for all domains.
Using the NumPy `datetime64` and `timedelta64` dtypes, pandas has consolidated a large number of
features from other Python libraries like `scikits.timeseries` as well as created
a tremendous amount of new functionality for manipulating time series data.

For example, pandas supports:

Parsing time series information from various sources and formats

Generate sequences of fixed-frequency dates and time spans

Manipulating and converting date times with timezone information

Resampling or converting a time series to a particular frequency

Performing date and time arithmetic with absolute or relative time increments

pandas provides a relatively compact and self-contained set of tools for
performing the above tasks and more.


<a id='timeseries-overview'></a>

## Overview

pandas captures 4 general time related concepts:

1. Date times: A specific date and time with timezone support. Similar to `datetime.datetime` from the standard library.  
1. Time deltas: An absolute time duration. Similar to `datetime.timedelta` from the standard library.  
1. Time spans: A span of time defined by a point in time and its associated frequency.  
1. Date offsets: A relative time duration that respects calendar arithmetic. Similar to `dateutil.relativedelta.relativedelta` from the `dateutil` package.  


````````````````````````````````````````|Concept|Scalar Class|Array Class|pandas Data Type|Primary Creation Method|
|:-------------------:|:---------------:|:-----------------:|:------------------------------------------:|:--------------------------------------:|
|Date times|Timestamp|DatetimeIndex|datetime64[ns] or datetime64[ns, tz]|to_datetime or date_range|
|Time deltas|Timedelta|TimedeltaIndex|timedelta64[ns]|to_timedelta or timedelta_range|
|Time spans|Period|PeriodIndex|period[freq]|Period or period_range|
|Date offsets|DateOffset|None|None|DateOffset|
For time series data, it’s conventional to represent the time component in the index of a `Series` or `DataFrame`
so manipulations can be performed with respect to the time element.

However, `Series` and `DataFrame` can directly also support the time component as data itself.

`Series` and `DataFrame` have extended data type support and functionality for `datetime`, `timedelta`
and `Period` data when passed into those constructors. `DateOffset`
data however will be stored as `object` data.

Lastly, pandas represents null date times, time deltas, and time spans as `NaT` which
is useful for representing missing or null date like values and behaves similar
as `np.nan` does for float data.


<a id='timeseries-representation'></a>

## Timestamps vs. Time Spans

Timestamped data is the most basic type of time series data that associates
values with points in time. For pandas objects it means using the points in
time.

However, in many cases it is more natural to associate things like change
variables with a time span instead. The span represented by `Period` can be
specified explicitly, or inferred from datetime string format.

For example:

`Timestamp` and `Period` can serve as an index. Lists of
`Timestamp` and `Period` are automatically coerced to `DatetimeIndex`
and `PeriodIndex` respectively.

pandas allows you to capture both representations and
convert between them. Under the hood, pandas represents timestamps using
instances of `Timestamp` and sequences of timestamps using instances of
`DatetimeIndex`. For regular time spans, pandas uses `Period` objects for
scalar values and `PeriodIndex` for sequences of spans. Better support for
irregular intervals with arbitrary start and end points are forth-coming in
future releases.


<a id='timeseries-converting'></a>

## Converting to timestamps

To convert a `Series` or list-like object of date-like objects e.g. strings,
epochs, or a mixture, you can use the `to_datetime` function. When passed
a `Series`, this returns a `Series` (with the same index), while a list-like
is converted to a `DatetimeIndex`:

If you use dates which start with the day first (i.e. European style),
you can pass the `dayfirst` flag:

You see in the above example that `dayfirst` isn’t strict, so if a date
can’t be parsed with the day being first it will be parsed as if
`dayfirst` were False.

If you pass a single string to `to_datetime`, it returns a single `Timestamp`.
`Timestamp` can also accept string input, but it doesn’t accept string parsing
options like `dayfirst` or `format`, so use `to_datetime` if these are required.

You can also use the `DatetimeIndex` constructor directly:

The string ‘infer’ can be passed in order to set the frequency of the index as the
inferred frequency upon creation:

### Providing a format argument

In addition to the required datetime string, a `format` argument can be passed to ensure specific parsing.
This could also potentially speed up the conversion considerably.

For more information on the choices available when specifying the `format`
option, see the Python [datetime documentation](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

### Assembling datetime from multiple DataFrame columns

New in version 0.18.1.

You can also pass a `DataFrame` of integer or string columns to assemble into a `Series` of `Timestamps`.

You can pass only the columns that you need to assemble.

`pd.to_datetime` looks for standard designations of the datetime component in the column names, including:

- required: `year`, `month`, `day`  
- optional: `hour`, `minute`, `second`, `millisecond`, `microsecond`, `nanosecond`  

### Invalid data

The default behavior, `errors='raise'`, is to raise when unparseable:

```ipython
In [2]: pd.to_datetime(['2009/07/31', 'asd'], errors='raise')
ValueError: Unknown string format
```


Pass `errors='ignore'` to return the original input when unparseable:

Pass `errors='coerce'` to convert unparseable data to `NaT` (not a time):


<a id='timeseries-converting-epoch'></a>

### Epoch timestamps

pandas supports converting integer or float epoch times to `Timestamp` and
`DatetimeIndex`. The default unit is nanoseconds, since that is how `Timestamp`
objects are stored internally. However, epochs are often stored in another `unit`
which can be specified. These are computed from the starting point specified by the
`origin` parameter.

Constructing a `Timestamp` or `DatetimeIndex` with an epoch timestamp
with the `tz` argument specified will currently localize the epoch timestamps to UTC
first then convert the result to the specified time zone. However, this behavior
is [deprecated](whatsnew/v0.24.0.ipynb#whatsnew-0240-deprecations-integer-tz), and if you have
epochs in wall time in another timezone, it is recommended to read the epochs
as timezone-naive timestamps and then localize to the appropriate timezone:

>**Note**
>
>Epoch times will be rounded to the nearest nanosecond.

Conversion of float epoch times can lead to inaccurate and unexpected results.
[Python floats](https://docs.python.org/3/tutorial/floatingpoint.html#tut-fp-issues) have about 15 digits precision in
decimal. Rounding during conversion from float to high precision `Timestamp` is
unavoidable. The only way to achieve exact precision is to use a fixed-width
types (e.g. an int64).

[Using the origin Parameter](#timeseries-origin)


<a id='timeseries-converting-epoch-inverse'></a>

### From timestamps to epoch

To invert the operation from above, namely, to convert from a `Timestamp` to a ‘unix’ epoch:

We subtract the epoch (midnight at January 1, 1970 UTC) and then floor divide by the
“unit” (1 second).


<a id='timeseries-origin'></a>

### Using the `origin` Parameter

New in version 0.20.0.

Using the `origin` parameter, one can specify an alternative starting point for creation
of a `DatetimeIndex`. For example, to use 1960-01-01 as the starting date:

The default is set at `origin='unix'`, which defaults to `1970-01-01 00:00:00`.
Commonly called ‘unix epoch’ or POSIX time.


<a id='timeseries-daterange'></a>

## Generating ranges of timestamps

To generate an index with timestamps, you can use either the `DatetimeIndex` or
`Index` constructor and pass in a list of datetime objects:

In practice this becomes very cumbersome because we often need a very long
index with a large number of timestamps. If we need timestamps on a regular
frequency, we can use the `date_range()` and `bdate_range()` functions
to create a `DatetimeIndex`. The default frequency for `date_range` is a
**calendar day** while the default for `bdate_range` is a **business day**:

Convenience functions like `date_range` and `bdate_range` can utilize a
variety of [frequency aliases](#timeseries-offset-aliases):

`date_range` and `bdate_range` make it easy to generate a range of dates
using various combinations of parameters like `start`, `end`, `periods`,
and `freq`. The start and end dates are strictly inclusive, so dates outside
of those specified will not be generated:

New in version 0.23.0.

Specifying `start`, `end`, and `periods` will generate a range of evenly spaced
dates from `start` to `end` inclusively, with `periods` number of elements in the
resulting `DatetimeIndex`:


<a id='timeseries-custom-freq-ranges'></a>

### Custom frequency ranges

`bdate_range` can also generate a range of custom frequency dates by using
the `weekmask` and `holidays` parameters.  These parameters will only be
used if a custom frequency string is passed.

[Custom business days](#timeseries-custombusinessdays)


<a id='timeseries-timestamp-limits'></a>

## Timestamp limitations

Since pandas represents timestamps in nanosecond resolution, the time span that
can be represented using a 64-bit integer is limited to approximately 584 years:

[Representing out-of-bounds spans](#timeseries-oob)


<a id='timeseries-datetimeindex'></a>

## Indexing

One of the main uses for `DatetimeIndex` is as an index for pandas objects.
The `DatetimeIndex` class contains many time series related optimizations:

- A large range of dates for various offsets are pre-computed and cached
  under the hood in order to make generating subsequent date ranges very fast
  (just have to grab a slice).  
- Fast shifting using the `shift` and `tshift` method on pandas objects.  
- Unioning of overlapping `DatetimeIndex` objects with the same frequency is
  very fast (important for fast data alignment).  
- Quick access to date fields via properties such as `year`, `month`, etc.  
- Regularization functions like `snap` and very fast `asof` logic.  


`DatetimeIndex` objects have all the basic functionality of regular `Index`
objects, and a smorgasbord of advanced time series specific methods for easy
frequency processing.

[Reindexing methods](getting_started/basics.ipynb#basics-reindexing)

>**Note**
>
>While pandas does not force you to have a sorted date index, some of these
methods may have unexpected or incorrect behavior if the dates are unsorted.

`DatetimeIndex` can be used like a regular index and offers all of its
intelligent functionality like selection, slicing, etc.


<a id='timeseries-partialindexing'></a>

### Partial string indexing

Dates and strings that parse to timestamps can be passed as indexing parameters:

To provide convenience for accessing longer time series, you can also pass in
the year or year and month as strings:

This type of slicing will work on a `DataFrame` with a `DatetimeIndex` as well. Since the
partial string selection is a form of label slicing, the endpoints **will be** included. This
would include matching times on an included date:

This starts on the very first time in the month, and includes the last date and
time for the month:

This specifies a stop time **that includes all of the times on the last day**:

This specifies an **exact** stop time (and is not the same as the above):

We are stopping on the included end-point as it is part of the index:

New in version 0.18.0.

`DatetimeIndex` partial string indexing also works on a `DataFrame` with a `MultiIndex`:

New in version 0.25.0.

Slicing with string indexing also honors UTC offset.


<a id='timeseries-slice-vs-exact-match'></a>

### Slice vs. exact match

Changed in version 0.20.0.

The same string used as an indexing parameter can be treated either as a slice or as an exact match depending on the resolution of the index. If the string is less accurate than the index, it will be treated as a slice, otherwise as an exact match.

Consider a `Series` object with a minute resolution index:

A timestamp string less accurate than a minute gives a `Series` object.

A timestamp string with minute resolution (or more accurate), gives a scalar instead, i.e. it is not casted to a slice.

If index resolution is second, then the minute-accurate timestamp gives a
`Series`.

If the timestamp string is treated as a slice, it can be used to index `DataFrame` with `[]` as well.

However, if the string is treated as an exact match, the selection in `DataFrame`’s `[]` will be column-wise and not row-wise, see [Indexing Basics](indexing.ipynb#indexing-basics). For example `dft_minute['2011-12-31 23:59']` will raise `KeyError` as `'2012-12-31 23:59'` has the same resolution as the index and there is no column with such name:

To *always* have unambiguous selection, whether the row is treated as a slice or a single selection, use `.loc`.

Note also that `DatetimeIndex` resolution cannot be less precise than day.

### Exact indexing

As discussed in previous section, indexing a `DatetimeIndex` with a partial string depends on the “accuracy” of the period, in other words how specific the interval is in relation to the resolution of the index. In contrast, indexing with `Timestamp` or `datetime` objects is exact, because the objects have exact meaning. These also follow the semantics of *including both endpoints*.

These `Timestamp` and `datetime` objects have exact `hours, minutes,` and `seconds`, even though they were not explicitly specified (they are `0`).

With no defaults.

### Truncating & fancy indexing

A `truncate()` convenience function is provided that is similar
to slicing. Note that `truncate` assumes a 0 value for any unspecified date
component in a `DatetimeIndex` in contrast to slicing which returns any
partially matching dates:

Even complicated fancy indexing that breaks the `DatetimeIndex` frequency
regularity will result in a `DatetimeIndex`, although frequency is lost:


<a id='timeseries-components'></a>

## Time/date components

There are several time/date properties that one can access from `Timestamp` or a collection of timestamps like a `DatetimeIndex`.

|Property|Description|
|:-------------:|:---------------------------------------------------------------:|
|year|The year of the datetime|
|month|The month of the datetime|
|day|The days of the datetime|
|hour|The hour of the datetime|
|minute|The minutes of the datetime|
|second|The seconds of the datetime|
|microsecond|The microseconds of the datetime|
|nanosecond|The nanoseconds of the datetime|
|date|Returns datetime.date (does not contain timezone information)|
|time|Returns datetime.time (does not contain timezone information)|
|timetz|Returns datetime.time as local time with timezone information|
|dayofyear|The ordinal day of year|
|weekofyear|The week ordinal of the year|
|week|The week ordinal of the year|
|dayofweek|The number of the day of the week with Monday=0, Sunday=6|
|weekday|The number of the day of the week with Monday=0, Sunday=6|
|weekday_name|The name of the day in a week (ex: Friday)|
|quarter|Quarter of the date: Jan-Mar = 1, Apr-Jun = 2, etc.|
|days_in_month|The number of days in the month of the datetime|
|is_month_start|Logical indicating if first day of month (defined by frequency)|
|is_month_end|Logical indicating if last day of month (defined by frequency)|
|is_quarter_start|Logical indicating if first day of quarter (defined by frequency)|
|is_quarter_end|Logical indicating if last day of quarter (defined by frequency)|
|is_year_start|Logical indicating if first day of year (defined by frequency)|
|is_year_end|Logical indicating if last day of year (defined by frequency)|
|is_leap_year|Logical indicating if the date belongs to a leap year|
Furthermore, if you have a `Series` with datetimelike values, then you can
access these properties via the `.dt` accessor, as detailed in the section
on [.dt accessors](getting_started/basics.ipynb#basics-dt-accessors).


<a id='timeseries-offsets'></a>

## DateOffset objects

In the preceding examples, frequency strings (e.g. `'D'`) were used to specify
a frequency that defined:

- how the date times in `DatetimeIndex` were spaced when using `date_range()`  
- the frequency of a `Period` or `PeriodIndex`  


These frequency strings map to a `DateOffset` object and its subclasses. A `DateOffset`
is similar to a `Timedelta` that represents a duration of time but follows specific calendar duration rules.
For example, a `Timedelta` day will always increment `datetimes` by 24 hours, while a `DateOffset` day
will increment `datetimes` to the same time the next day whether a day represents 23, 24 or 25 hours due to daylight
savings time. However, all `DateOffset` subclasses that are an hour or smaller
(`Hour`, `Minute`, `Second`, `Milli`, `Micro`, `Nano`) behave like
`Timedelta` and respect absolute time.

The basic `DateOffset` acts similar to `dateutil.relativedelta` ([relativedelta documentation](https://dateutil.readthedocs.io/en/stable/relativedelta.html))
that shifts a date time by the corresponding calendar duration specified. The
arithmetic operator (`+`) or the `apply` method can be used to perform the shift.

Most `DateOffsets` have associated frequencies strings, or offset aliases, that can be passed
into `freq` keyword arguments. The available date offsets and associated frequency strings can be found below:

````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````|Date Offset|Frequency String|Description|
|:-------------:|:-------------:|:---------------------------------------------------------------:|
|DateOffset|None|Generic offset class, defaults to 1 calendar day|
|BDay or BusinessDay|'B'|business day (weekday)|
|CDay or CustomBusinessDay|'C'|custom business day|
|Week|'W'|one week, optionally anchored on a day of the week|
|WeekOfMonth|'WOM'|the x-th day of the y-th week of each month|
|LastWeekOfMonth|'LWOM'|the x-th day of the last week of each month|
|MonthEnd|'M'|calendar month end|
|MonthBegin|'MS'|calendar month begin|
|BMonthEnd or BusinessMonthEnd|'BM'|business month end|
|BMonthBegin or BusinessMonthBegin|'BMS'|business month begin|
|CBMonthEnd or CustomBusinessMonthEnd|'CBM'|custom business month end|
|CBMonthBegin or CustomBusinessMonthBegin|'CBMS'|custom business month begin|
|SemiMonthEnd|'SM'|15th (or other day_of_month) and calendar month end|
|SemiMonthBegin|'SMS'|15th (or other day_of_month) and calendar month begin|
|QuarterEnd|'Q'|calendar quarter end|
|QuarterBegin|'QS'|calendar quarter begin|
|BQuarterEnd|'BQ|business quarter end|
|BQuarterBegin|'BQS'|business quarter begin|
|FY5253Quarter|'REQ'|retail (aka 52-53 week) quarter|
|YearEnd|'A'|calendar year end|
|YearBegin|'AS' or 'BYS'|calendar year begin|
|BYearEnd|'BA'|business year end|
|BYearBegin|'BAS'|business year begin|
|FY5253|'RE'|retail (aka 52-53 week) year|
|Easter|None|Easter holiday|
|BusinessHour|'BH'|business hour|
|CustomBusinessHour|'CBH'|custom business hour|
|Day|'D'|one absolute day|
|Hour|'H'|one hour|
|Minute|'T' or 'min'|one minute|
|Second|'S'|one second|
|Milli|'L' or 'ms'|one millisecond|
|Micro|'U' or 'us'|one microsecond|
|Nano|'N'|one nanosecond|
`DateOffsets` additionally have `rollforward()` and `rollback()`
methods for moving a date forward or backward respectively to a valid offset
date relative to the offset. For example, business offsets will roll dates
that land on the weekends (Saturday and Sunday) forward to Monday since
business offsets operate on the weekdays.

These operations preserve time (hour, minute, etc) information by default.
To reset time to midnight, use `normalize()` before or after applying
the operation (depending on whether you want the time information included
in the operation).

### Parametric offsets

Some of the offsets can be “parameterized” when created to result in different
behaviors. For example, the `Week` offset for generating weekly data accepts a
`weekday` parameter which results in the generated dates always lying on a
particular day of the week:

The `normalize` option will be effective for addition and subtraction.

Another example is parameterizing `YearEnd` with the specific ending month:


<a id='timeseries-offsetseries'></a>

### Using offsets with `Series` / `DatetimeIndex`

Offsets can be used with either a `Series` or `DatetimeIndex` to
apply the offset to each element.

If the offset class maps directly to a `Timedelta` (`Day`, `Hour`,
`Minute`, `Second`, `Micro`, `Milli`, `Nano`) it can be
used exactly like a `Timedelta` - see the
[Timedelta section](timedeltas.ipynb#timedeltas-operations) for more examples.

Note that some offsets (such as `BQuarterEnd`) do not have a
vectorized implementation.  They can still be used but may
calculate significantly slower and will show a `PerformanceWarning`


<a id='timeseries-custombusinessdays'></a>

### Custom business days

The `CDay` or `CustomBusinessDay` class provides a parametric
`BusinessDay` class which can be used to create customized business day
calendars which account for local holidays and local weekend conventions.

As an interesting example, let’s look at Egypt where a Friday-Saturday weekend is observed.

Let’s map to the weekday names:

Holiday calendars can be used to provide the list of holidays.  See the
[holiday calendar](#timeseries-holiday) section for more information.

Monthly offsets that respect a certain holiday calendar can be defined
in the usual way.

>**Note**
>
>The frequency string ‘C’ is used to indicate that a CustomBusinessDay
DateOffset is used, it is important to note that since CustomBusinessDay is
a parameterised type, instances of CustomBusinessDay may differ and this is
not detectable from the ‘C’ frequency string. The user therefore needs to
ensure that the ‘C’ frequency string is used consistently within the user’s
application.


<a id='timeseries-businesshour'></a>

### Business hour

The `BusinessHour` class provides a business hour representation on `BusinessDay`,
allowing to use specific start and end times.

By default, `BusinessHour` uses 9:00 - 17:00 as business hours.
Adding `BusinessHour` will increment `Timestamp` by hourly frequency.
If target `Timestamp` is out of business hours, move to the next business hour
then increment it. If the result exceeds the business hours end, the remaining
hours are added to the next business day.

You can also specify `start` and `end` time by keywords. The argument must
be a `str` with an `hour:minute` representation or a `datetime.time`
instance. Specifying seconds, microseconds and nanoseconds as business hour
results in `ValueError`.

Passing `start` time later than `end` represents midnight business hour.
In this case, business hour exceeds midnight and overlap to the next day.
Valid business hours are distinguished by whether it started from valid `BusinessDay`.

Applying `BusinessHour.rollforward` and `rollback` to out of business hours results in
the next business hour start or previous day’s end. Different from other offsets, `BusinessHour.rollforward`
may output different results from `apply` by definition.

This is because one day’s business hour end is equal to next day’s business hour start. For example,
under the default business hours (9:00 - 17:00), there is no gap (0 minutes) between `2014-08-01 17:00` and
`2014-08-04 09:00`.

`BusinessHour` regards Saturday and Sunday as holidays. To use arbitrary
holidays, you can use `CustomBusinessHour` offset, as explained in the
following subsection.


<a id='timeseries-custombusinesshour'></a>

### Custom business hour

New in version 0.18.1.

The `CustomBusinessHour` is a mixture of `BusinessHour` and `CustomBusinessDay` which
allows you to specify arbitrary holidays. `CustomBusinessHour` works as the same
as `BusinessHour` except that it skips specified custom holidays.

You can use keyword arguments supported by either `BusinessHour` and `CustomBusinessDay`.


<a id='timeseries-offset-aliases'></a>

### Offset aliases

A number of string aliases are given to useful common time series
frequencies. We will refer to these aliases as *offset aliases*.

|Alias|Description|
|:-------------:|:--------------------------------------------------------------------------------------------------:|
|B|business day frequency|
|C|custom business day frequency|
|D|calendar day frequency|
|W|weekly frequency|
|M|month end frequency|
|SM|semi-month end frequency (15th and end of month)|
|BM|business month end frequency|
|CBM|custom business month end frequency|
|MS|month start frequency|
|SMS|semi-month start frequency (1st and 15th)|
|BMS|business month start frequency|
|CBMS|custom business month start frequency|
|Q|quarter end frequency|
|BQ|business quarter end frequency|
|QS|quarter start frequency|
|BQS|business quarter start frequency|
|A, Y|year end frequency|
|BA, BY|business year end frequency|
|AS, YS|year start frequency|
|BAS, BYS|business year start frequency|
|BH|business hour frequency|
|H|hourly frequency|
|T, min|minutely frequency|
|S|secondly frequency|
|L, ms|milliseconds|
|U, us|microseconds|
|N|nanoseconds|

### Combining aliases

As we have seen previously, the alias and the offset instance are fungible in
most functions:

You can combine together day and intraday offsets:

### Anchored offsets

For some frequencies you can specify an anchoring suffix:

|Alias|Description|
|:-------------:|:--------------------------------------------------------------------------------------------------:|
|W-SUN|weekly frequency (Sundays). Same as ‘W’|
|W-MON|weekly frequency (Mondays)|
|W-TUE|weekly frequency (Tuesdays)|
|W-WED|weekly frequency (Wednesdays)|
|W-THU|weekly frequency (Thursdays)|
|W-FRI|weekly frequency (Fridays)|
|W-SAT|weekly frequency (Saturdays)|
|(B)Q(S)-DEC|quarterly frequency, year ends in December. Same as ‘Q’|
|(B)Q(S)-JAN|quarterly frequency, year ends in January|
|(B)Q(S)-FEB|quarterly frequency, year ends in February|
|(B)Q(S)-MAR|quarterly frequency, year ends in March|
|(B)Q(S)-APR|quarterly frequency, year ends in April|
|(B)Q(S)-MAY|quarterly frequency, year ends in May|
|(B)Q(S)-JUN|quarterly frequency, year ends in June|
|(B)Q(S)-JUL|quarterly frequency, year ends in July|
|(B)Q(S)-AUG|quarterly frequency, year ends in August|
|(B)Q(S)-SEP|quarterly frequency, year ends in September|
|(B)Q(S)-OCT|quarterly frequency, year ends in October|
|(B)Q(S)-NOV|quarterly frequency, year ends in November|
|(B)A(S)-DEC|annual frequency, anchored end of December. Same as ‘A’|
|(B)A(S)-JAN|annual frequency, anchored end of January|
|(B)A(S)-FEB|annual frequency, anchored end of February|
|(B)A(S)-MAR|annual frequency, anchored end of March|
|(B)A(S)-APR|annual frequency, anchored end of April|
|(B)A(S)-MAY|annual frequency, anchored end of May|
|(B)A(S)-JUN|annual frequency, anchored end of June|
|(B)A(S)-JUL|annual frequency, anchored end of July|
|(B)A(S)-AUG|annual frequency, anchored end of August|
|(B)A(S)-SEP|annual frequency, anchored end of September|
|(B)A(S)-OCT|annual frequency, anchored end of October|
|(B)A(S)-NOV|annual frequency, anchored end of November|
These can be used as arguments to `date_range`, `bdate_range`, constructors
for `DatetimeIndex`, as well as various other timeseries-related functions
in pandas.

### Anchored offset semantics

For those offsets that are anchored to the start or end of specific
frequency (`MonthEnd`, `MonthBegin`, `WeekEnd`, etc), the following
rules apply to rolling forward and backwards.

When `n` is not 0, if the given date is not on an anchor point, it snapped to the next(previous)
anchor point, and moved `|n|-1` additional steps forwards or backwards.

If the given date *is* on an anchor point, it is moved `|n|` points forwards
or backwards.

For the case when `n=0`, the date is not moved if on an anchor point, otherwise
it is rolled forward to the next anchor point.


<a id='timeseries-holiday'></a>

### Holidays / holiday calendars

Holidays and calendars provide a simple way to define holiday rules to be used
with `CustomBusinessDay` or in other analysis that requires a predefined
set of holidays.  The `AbstractHolidayCalendar` class provides all the necessary
methods to return a list of holidays and only `rules` need to be defined
in a specific holiday calendar class. Furthermore, the `start_date` and `end_date`
class attributes determine over what date range holidays are generated.  These
should be overwritten on the `AbstractHolidayCalendar` class to have the range
apply to all calendar subclasses.  `USFederalHolidayCalendar` is the
only calendar that exists and primarily serves as an example for developing
other calendars.

For holidays that occur on fixed dates (e.g., US Memorial Day or July 4th) an
observance rule determines when that holiday is observed if it falls on a weekend
or some other non-observed day.  Defined observance rules are:

|Rule|Description|
|:-------------:|:--------------------------------------------------------------------:|
|nearest_workday|move Saturday to Friday and Sunday to Monday|
|sunday_to_monday|move Sunday to following Monday|
|next_monday_or_tuesday|move Saturday to Monday and Sunday/Monday to Tuesday|
|previous_friday|move Saturday and Sunday to previous Friday”|
|next_monday|move Saturday and Sunday to following Monday|
An example of how holidays and holiday calendars are defined:


<dl style='margin: 20px 0;'>
<dt>hint</dt>
<dd>
**weekday=MO(2)** is same as **2 * Week(weekday=2)**

</dd>

</dl>

Using this calendar, creating an index or doing offset arithmetic skips weekends
and holidays (i.e., Memorial Day/July 4th).  For example, the below defines
a custom business day offset using the `ExampleCalendar`.  Like any other offset,
it can be used to create a `DatetimeIndex` or added to `datetime`
or `Timestamp` objects.

Ranges are defined by the `start_date` and `end_date` class attributes
of `AbstractHolidayCalendar`.  The defaults are shown below.

These dates can be overwritten by setting the attributes as
datetime/Timestamp/string.

Every calendar class is accessible by name using the `get_calendar` function
which returns a holiday class instance.  Any imported calendar class will
automatically be available by this function.  Also, `HolidayCalendarFactory`
provides an easy interface to create calendars that are combinations of calendars
or calendars with additional rules.


<a id='timeseries-advanced-datetime'></a>

## Time Series-Related Instance Methods

### Shifting / lagging

One may want to *shift* or *lag* the values in a time series back and forward in
time. The method for this is `shift()`, which is available on all of
the pandas objects.

The `shift` method accepts an `freq` argument which can accept a
`DateOffset` class or other `timedelta`-like object or also an
[offset alias](#timeseries-offset-aliases):

Rather than changing the alignment of the data and the index, `DataFrame` and
`Series` objects also have a `tshift()` convenience method that
changes all the dates in the index by a specified number of offsets:

Note that with `tshift`, the leading entry is no longer NaN because the data
is not being realigned.

### Frequency conversion

The primary function for changing frequencies is the `asfreq()`
method. For a `DatetimeIndex`, this is basically just a thin, but convenient
wrapper around `reindex()`  which generates a `date_range` and
calls `reindex`.

`asfreq` provides a further convenience so you can specify an interpolation
method for any gaps that may appear after the frequency conversion.

### Filling forward / backward

Related to `asfreq` and `reindex` is `fillna()`, which is
documented in the [missing data section](missing_data.ipynb#missing-data-fillna).

### Converting to Python datetimes

`DatetimeIndex` can be converted to an array of Python native
[`datetime.datetime`](https://docs.python.org/3/library/datetime.html#datetime.datetime) objects using the `to_pydatetime` method.


<a id='timeseries-resampling'></a>

## Resampling

The interface to `.resample` has changed in 0.18.0 to be more groupby-like and hence more flexible.
See the [whatsnew docs](whatsnew/v0.18.0.ipynb#whatsnew-0180-breaking-resample) for a comparison with prior versions.

Pandas has a simple, powerful, and efficient functionality for performing
resampling operations during frequency conversion (e.g., converting secondly
data into 5-minutely data). This is extremely common in, but not limited to,
financial applications.

`resample()` is a time-based groupby, followed by a reduction method
on each of its groups. See some [cookbook examples](cookbook.ipynb#cookbook-resample) for
some advanced strategies.

Starting in version 0.18.1, the `resample()` function can be used directly from
`DataFrameGroupBy` objects, see the [groupby docs](groupby.ipynb#groupby-transform-window-resample).

>**Note**
>
>`.resample()` is similar to using a `rolling()` operation with
a time-based offset, see a discussion [here](computation.ipynb#stats-moments-ts-versus-resampling).

### Basics

The `resample` function is very flexible and allows you to specify many
different parameters to control the frequency conversion and resampling
operation.

Any function available via [dispatching](groupby.ipynb#groupby-dispatch) is available as
a method of the returned object, including `sum`, `mean`, `std`, `sem`,
`max`, `min`, `median`, `first`, `last`, `ohlc`:

For downsampling, `closed` can be set to ‘left’ or ‘right’ to specify which
end of the interval is closed:

Parameters like `label` and `loffset` are used to manipulate the resulting
labels. `label` specifies whether the result is labeled with the beginning or
the end of the interval. `loffset` performs a time adjustment on the output
labels.

The default values for `label` and `closed` is ‘**left**’ for all
frequency offsets except for ‘M’, ‘A’, ‘Q’, ‘BM’, ‘BA’, ‘BQ’, and ‘W’
which all have a default of ‘right’.

This might unintendedly lead to looking ahead, where the value for a later
time is pulled back to a previous time as in the following example with
the `BusinessDay` frequency:

Notice how the value for Sunday got pulled back to the previous Friday.
To get the behavior where the value for Sunday is pushed to Monday, use
instead

The `axis` parameter can be set to 0 or 1 and allows you to resample the
specified axis for a `DataFrame`.

`kind` can be set to ‘timestamp’ or ‘period’ to convert the resulting index
to/from timestamp and time span representations. By default `resample`
retains the input representation.

`convention` can be set to ‘start’ or ‘end’ when resampling period data
(detail below). It specifies how low frequency periods are converted to higher
frequency periods.

### Upsampling

For upsampling, you can specify a way to upsample and the `limit` parameter to interpolate over the gaps that are created:

### Sparse resampling

Sparse timeseries are the ones where you have a lot fewer points relative
to the amount of time you are looking to resample. Naively upsampling a sparse
series can potentially generate lots of intermediate values. When you don’t want
to use a method to fill these values, e.g. `fill_method` is `None`, then
intermediate values will be filled with `NaN`.

Since `resample` is a time-based groupby, the following is a method to efficiently
resample only the groups that are not all `NaN`.

If we want to resample to the full range of the series:

We can instead only resample those groups where we have points as follows:


<a id='timeseries-aggregate'></a>

### Aggregation

Similar to the [aggregating API](getting_started/basics.ipynb#basics-aggregate), [groupby API](groupby.ipynb#groupby-aggregate), and the [window functions API](computation.ipynb#stats-aggregate),
a `Resampler` can be selectively resampled.

Resampling a `DataFrame`, the default will be to act on all columns with the same function.

We can select a specific column or columns using standard getitem.

You can pass a list or dict of functions to do aggregation with, outputting a `DataFrame`:

On a resampled `DataFrame`, you can pass a list of functions to apply to each
column, which produces an aggregated result with a hierarchical index:

By passing a dict to `aggregate` you can apply a different aggregation to the
columns of a `DataFrame`:

The function names can also be strings. In order for a string to be valid it
must be implemented on the resampled object:

Furthermore, you can also specify multiple aggregation functions for each column separately.

If a `DataFrame` does not have a datetimelike index, but instead you want
to resample based on datetimelike column in the frame, it can passed to the
`on` keyword.

Similarly, if you instead want to resample by a datetimelike
level of `MultiIndex`, its name or location can be passed to the
`level` keyword.


<a id='timeseries-iterating-label'></a>

### Iterating through groups

With the `Resampler` object in hand, iterating through the grouped data is very
natural and functions similarly to [`itertools.groupby()`](https://docs.python.org/3/library/itertools.html#itertools.groupby):

See [Iterating through groups](groupby.ipynb#groupby-iterating-label) or `Resampler.__iter__` for more.


<a id='timeseries-periods'></a>

## Time span representation

Regular intervals of time are represented by `Period` objects in pandas while
sequences of `Period` objects are collected in a `PeriodIndex`, which can
be created with the convenience function `period_range`.

### Period

A `Period` represents a span of time (e.g., a day, a month, a quarter, etc).
You can specify the span via `freq` keyword using a frequency alias like below.
Because `freq` represents a span of `Period`, it cannot be negative like “-3D”.

Adding and subtracting integers from periods shifts the period by its own
frequency. Arithmetic is not allowed between `Period` with different `freq` (span).

If `Period` freq is daily or higher (`D`, `H`, `T`, `S`, `L`, `U`, `N`), `offsets` and `timedelta`-like can be added if the result can have the same freq. Otherwise, `ValueError` will be raised.

```ipython
In [1]: p + pd.offsets.Minute(5)
Traceback
   ...
ValueError: Input has different freq from Period(freq=H)
```


If `Period` has other frequencies, only the same `offsets` can be added. Otherwise, `ValueError` will be raised.

```ipython
In [1]: p + pd.offsets.MonthBegin(3)
Traceback
   ...
ValueError: Input has different freq from Period(freq=M)
```


Taking the difference of `Period` instances with the same frequency will
return the number of frequency units between them:

### PeriodIndex and period_range

Regular sequences of `Period` objects can be collected in a `PeriodIndex`,
which can be constructed using the `period_range` convenience function:

The `PeriodIndex` constructor can also be used directly:

Passing multiplied frequency outputs a sequence of `Period` which
has multiplied span.

If `start` or `end` are `Period` objects, they will be used as anchor
endpoints for a `PeriodIndex` with frequency matching that of the
`PeriodIndex` constructor.

Just like `DatetimeIndex`, a `PeriodIndex` can also be used to index pandas
objects:

`PeriodIndex` supports addition and subtraction with the same rule as `Period`.

`PeriodIndex` has its own dtype named `period`, refer to [Period Dtypes](#timeseries-period-dtype).


<a id='timeseries-period-dtype'></a>

### Period dtypes

New in version 0.19.0.

`PeriodIndex` has a custom `period` dtype. This is a pandas extension
dtype similar to the [timezone aware dtype](#timeseries-timezone-series) (`datetime64[ns, tz]`).

The `period` dtype holds the `freq` attribute and is represented with
`period[freq]` like `period[D]` or `period[M]`, using [frequency strings](#timeseries-offset-aliases).

The `period` dtype can be used in `.astype(...)`. It allows one to change the
`freq` of a `PeriodIndex` like `.asfreq()` and convert a
`DatetimeIndex` to `PeriodIndex` like `to_period()`:

### PeriodIndex partial string indexing

You can pass in dates and strings to `Series` and `DataFrame` with `PeriodIndex`, in the same manner as `DatetimeIndex`. For details, refer to [DatetimeIndex Partial String Indexing](#timeseries-partialindexing).

Passing a string representing a lower frequency than `PeriodIndex` returns partial sliced data.

As with `DatetimeIndex`, the endpoints will be included in the result. The example below slices data starting from 10:00 to 11:59.

### Frequency conversion and resampling with PeriodIndex

The frequency of `Period` and `PeriodIndex` can be converted via the `asfreq`
method. Let’s start with the fiscal year 2011, ending in December:

We can convert it to a monthly frequency. Using the `how` parameter, we can
specify whether to return the starting or ending month:

The shorthands ‘s’ and ‘e’ are provided for convenience:

Converting to a “super-period” (e.g., annual frequency is a super-period of
quarterly frequency) automatically returns the super-period that includes the
input period:

Note that since we converted to an annual frequency that ends the year in
November, the monthly period of December 2011 is actually in the 2012 A-NOV
period.


<a id='timeseries-quarterly'></a>
Period conversions with anchored frequencies are particularly useful for
working with various quarterly data common to economics, business, and other
fields. Many organizations define quarters relative to the month in which their
fiscal year starts and ends. Thus, first quarter of 2011 could start in 2010 or
a few months into 2011. Via anchored frequencies, pandas works for all quarterly
frequencies `Q-JAN` through `Q-DEC`.

`Q-DEC` define regular calendar quarters:

`Q-MAR` defines fiscal year end in March:


<a id='timeseries-interchange'></a>

## Converting between representations

Timestamped data can be converted to PeriodIndex-ed data using `to_period`
and vice-versa using `to_timestamp`:

Remember that ‘s’ and ‘e’ can be used to return the timestamps at the start or
end of the period:

Converting between period and timestamp enables some convenient arithmetic
functions to be used. In the following example, we convert a quarterly
frequency with year ending in November to 9am of the end of the month following
the quarter end:


<a id='timeseries-oob'></a>

## Representing out-of-bounds spans

If you have data that is outside of the `Timestamp` bounds, see [Timestamp limitations](#timeseries-timestamp-limits),
then you can use a `PeriodIndex` and/or `Series` of `Periods` to do computations.

To convert from an `int64` based YYYYMMDD representation.

These can easily be converted to a `PeriodIndex`:


<a id='timeseries-timezone'></a>

## Time zone handling

pandas provides rich support for working with timestamps in different time
zones using the `pytz` and `dateutil` libraries or class:datetime.timezone
objects from the standard library.

### Working with time zones

By default, pandas objects are time zone unaware:

To localize these dates to a time zone (assign a particular time zone to a naive date),
you can use the `tz_localize` method or the `tz` keyword argument in
`date_range()`, `Timestamp`, or `DatetimeIndex`.
You can either pass `pytz` or `dateutil` time zone objects or Olson time zone database strings.
Olson time zone strings will return `pytz` time zone objects by default.
To return `dateutil` time zone objects, append `dateutil/` before the string.

- In `pytz` you can find a list of common (and less common) time zones using
  `from pytz import common_timezones, all_timezones`.  
- `dateutil` uses the OS time zones so there isn’t a fixed list available. For
  common zones, the names are the same as `pytz`.  


New in version 0.25.0.

Note that the `UTC` time zone is a special case in `dateutil` and should be constructed explicitly
as an instance of `dateutil.tz.tzutc`. You can also construct other time
zones objects explicitly first.

To convert a time zone aware pandas object from one time zone to another,
you can use the `tz_convert` method.

>**Note**
>
>When using `pytz` time zones, `DatetimeIndex` will construct a different
time zone object than a `Timestamp` for the same time zone input. A `DatetimeIndex`
can hold a collection of `Timestamp` objects that may have different UTC offsets and cannot be
succinctly represented by one `pytz` time zone instance while one `Timestamp`
represents one point in time with a specific UTC offset.

Be wary of conversions between libraries. For some time zones, `pytz` and `dateutil` have different
definitions of the zone. This is more of a problem for unusual time zones than for
‘standard’ zones like `US/Eastern`.

Be aware that a time zone definition across versions of time zone libraries may not
be considered equal.  This may cause problems when working with stored data that
is localized using one version and operated on with a different version.
See [here](io.ipynb#io-hdf5-notes) for how to handle such a situation.

For `pytz` time zones, it is incorrect to pass a time zone object directly into
the `datetime.datetime` constructor
(e.g., `datetime.datetime(2011, 1, 1, tz=pytz.timezone('US/Eastern'))`.
Instead, the datetime needs to be localized using the `localize` method
on the `pytz` time zone object.

Under the hood, all timestamps are stored in UTC. Values from a time zone aware
`DatetimeIndex` or `Timestamp` will have their fields (day, hour, minute, etc.)
localized to the time zone. However, timestamps with the same UTC value are
still considered to be equal even if they are in different time zones:

Operations between `Series` in different time zones will yield UTC
`Series`, aligning the data on the UTC timestamps:

To remove time zone information, use `tz_localize(None)` or `tz_convert(None)`.
`tz_localize(None)` will remove the time zone yielding the local time representation.
`tz_convert(None)` will remove the time zone after converting to UTC time.


<a id='timeseries-timezone-ambiguous'></a>

### Ambiguous times when localizing

`tz_localize` may not be able to determine the UTC offset of a timestamp
because daylight savings time (DST) in a local time zone causes some times to occur
twice within one day (“clocks fall back”). The following options are available:

- `'raise'`: Raises a `pytz.AmbiguousTimeError` (the default behavior)  
- `'infer'`: Attempt to determine the correct offset base on the monotonicity of the timestamps  
- `'NaT'`: Replaces ambiguous times with `NaT`  
- `bool`: `True` represents a DST time, `False` represents non-DST time. An array-like of `bool` values is supported for a sequence of times.  


This will fail as there are ambiguous times (`'11/06/2011 01:00'`)

```ipython
In [2]: rng_hourly.tz_localize('US/Eastern')
AmbiguousTimeError: Cannot infer dst time from Timestamp('2011-11-06 01:00:00'), try using the 'ambiguous' argument
```


Handle these ambiguous times by specifying the following.


<a id='timeseries-timezone-nonexistent'></a>

### Nonexistent times when localizing

A DST transition may also shift the local time ahead by 1 hour creating nonexistent
local times (“clocks spring forward”). The behavior of localizing a timeseries with nonexistent times
can be controlled by the `nonexistent` argument. The following options are available:

- `'raise'`: Raises a `pytz.NonExistentTimeError` (the default behavior)  
- `'NaT'`: Replaces nonexistent times with `NaT`  
- `'shift_forward'`: Shifts nonexistent times forward to the closest real time  
- `'shift_backward'`: Shifts nonexistent times backward to the closest real time  
- timedelta object: Shifts nonexistent times by the timedelta duration  


Localization of nonexistent times will raise an error by default.

```ipython
In [2]: dti.tz_localize('Europe/Warsaw')
NonExistentTimeError: 2015-03-29 02:30:00
```


Transform nonexistent times to `NaT` or shift the times.


<a id='timeseries-timezone-series'></a>

### Time zone series operations

A `Series` with time zone **naive** values is
represented with a dtype of `datetime64[ns]`.

A `Series` with a time zone **aware** values is
represented with a dtype of `datetime64[ns, tz]` where `tz` is the time zone

Both of these `Series` time zone information
can be manipulated via the `.dt` accessor, see [the dt accessor section](getting_started/basics.ipynb#basics-dt-accessors).

For example, to localize and convert a naive stamp to time zone aware.

Time zone information can also be manipulated using the `astype` method.
This method can localize and convert time zone naive timestamps or
convert time zone aware timestamps.

>**Note**
>
>Using `Series.to_numpy()` on a `Series`, returns a NumPy array of the data.
NumPy does not currently support time zones (even though it is *printing* in the local time zone!),
therefore an object array of Timestamps is returned for time zone aware data:

By converting to an object array of Timestamps, it preserves the time zone
information. For example, when converting back to a Series:

However, if you want an actual NumPy `datetime64[ns]` array (with the values
converted to UTC) instead of an array of objects, you can specify the
`dtype` argument: